<h1> Stability of the Italian Government </h1>
<h3> Davide Cremonini </h3> 
<h3>Foundamentals of Artificial Intelligence - Module 3 Project </h3>

<p>This project aims at the definition of a Bayesian Network which captures the causal links between purely statistichal data of past Italian governments in an attempt to predict instability conditions for the future ones.</p>

In [1]:
#First we import every library which will be used
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination, BeliefPropagation

In [2]:
#Reading and showing the dataset, performing NaN rows cleanup
df = pd.read_csv("Dataset_Governi_OK.csv", sep=';', decimal=',', true_values=['T', 'P'], false_values=['F'])
df=df.dropna(axis=0, how='all')
df.head()

,Gov Name,Time in Charge,Effective Time in Charge,House Majority,House Opposition,Senate Majority,Senate Opposition,N° of VP,N° Ministers,N° Ministers without portfolio,...,First Year,Legislation,Confidence,Fall Cause,PM from Largest Majority Party,Technocrat/Independent PM,N° Majority Parties,N° Opposition Parties,DI Parties in Government,Tot Gov. Members
0,De Gasperi V,613.0,598.0,369.0,205.0,156.0,237.0,3.0,17.0,2.0,...,1948.0,1.0,True,External Pressure,True,False,5.0,5.0,0.65,23.0
1,De Gasperi VI,545.0,535.0,336.0,238.0,149.0,88.0,0.0,17.0,3.0,...,1950.0,1.0,True,Internal Struggle,True,False,4.0,7.0,0.48,21.0
2,De Gasperi VII,721.0,704.0,317.0,257.0,139.0,98.0,1.0,17.0,2.0,...,1951.0,1.0,True,End of Legislation,True,False,3.0,7.0,0.26,21.0
3,De Gasperi VIII,32.0,12.0,263.0,327.0,116.0,12.0,1.0,16.0,2.0,...,1953.0,2.0,False,No confidence,True,False,1.0,8.0,0.00,20.0
4,Pella,155.0,141.0,324.0,266.0,141.0,102.0,0.0,16.0,2.0,...,1953.0,2.0,True,Internal Struggle,True,False,5.0,4.0,0.00,19.0


<h4>The dataset</h4>
<div>The dataset contains data on all of the 64 governments that lead Italy from the First Legislation to the Eighteenth.
This dataset was built by myself using the information found on the following sitse:<br>
    https://www.governo.it/it/i-governi-dal-1943-ad-oggi/191<br>
    https://www.senato.it/legislature/repubblica/governi-della-repubblica<br>
    https://it.wikipedia.org/wiki/Template:Governi_della_Repubblica_Italiana<br>
</div><br>
<div>
There are a total of 26 columns in the dataset:
<ol>
    <li><b>Gov Name</b>, the official government name, which derives from the surname of the Prime Minister.</li>
    <li><b>Time in Charge</b>, the number of days the government officially held power from its oath of office to the next one.</li>
    <li><b>Effective Time in Charge</b>, the number of days the government was actually in charge, from its oath of office to the resignation of the Prime Minister</li>
    <li><b>House Majority</b>, the number of House of Representatives voters which gave confidence to the government at the beginning of its office.</li>
    <li><b>House Opposition</b>, the number of House of Representatives voters which denied confidence or abstained to the government at the beginning of its office.</li>
    <li><b>Senate Majority</b>, the number of Senate of the Republic voters which gave confidence to the government at the beginning of its office.</li>
    <li><b>Senate Opposition</b>, the number of Senate of the Republic voters which denied confidence or abstained to the government at the beginning of its office.</li>
    <li><b>N° of VP</b>, the number of Vice-Prime Ministers each government had, it's mostly an honorific title given to the leaders of a lesser coalition party.</li>
    <li><b>N° of Ministers</b>, the number of Ministers each government had, they are usually present in every government, regardless of the political agenda.</li>
    <li><b>N° of Ministers without portfolio</b>, the number of Ministers without Portfolio each government had, theier roles change in every government as they are an expression of the political agenda of said government.</li>
    <li><b>Swapped Ministers</b>, the number of times a minister was changed during the office, this may have happened for different reasons, like resignation or death of said minister.</li>
    <li><b>DI House Majority</b>, the majority's diversity in the House of Representatives when the government asked for confidence. It is expressed according to Simpson's Diversity index, which takes into consideration each party as a different species and the number of voters it contained as the frequency for each species.</li>
    <li><b>DI House Opposition</b>, the opposition's diversity in the House of the Representatives, according to the SDI.</li>
    <li><b>DI Senate Majority</b>, the majority's diversity in the Senate of the Republic, according to the SDI.</li>
    <li><b>DI Senate Opposition</b>, the opposition's diversity in the Senate of the Republic, according to the SDI.</li>
    <li><b>End of the Legislation</b>, a boolean value which expresses whether or not one government was the last of its legislation.</li>
    <li><b>First Year</b>, the year number in which the government swore office.</li>
    <li><b>Legislation</b>, the number of parliamentary legislation which was present when the governement took office.</li>
    <li><b>Confidence</b>, a boolean which expresses whether or not each government gained confidence from both houses of the Parliament.</li>
    <li><b>Fall Reason</b>, a summarized short description which tries to explain which was the main cause for the government fall.</li>
    <li><b>PM from Largest Majority Party</b>, a boolean which expresses whether or not each government's PM was a member of the largest party of the coalition.</li>
    <li><b>Technocrat/Independent PM</b>, a boolean which expresses whether or not the Prime Minister of each government was not a member of any of parties inside the coalition (Independent) or, in case he was completely external from the political activity, a so called 'Technocrat PM'.</li>
    <li><b>N° Majority Parties</b>, the number of parties which mostly voted for giving confidence to the government.</li>
    <li><b>N° Opposition Parties</b>, the number of parties which denied or abstained from giving confidence to the government.</li>
    <li><b>DI Parties in Government</b>, party diversity inside the government seats, according to SDI using the parties as species and the numbers of PM, and Ministers as frequency.</li>
    <li><b>Tot Gov. Members</b>, total number of government members, a sum of values included in 'N° of VP', 'N° of Ministers' and 'N° of Ministers without Portfolio', plus one for the Prime Minister</li>
</ol>
</div>

In [3]:
#Pre-prorcessing Actvities

#Changing Majority and Opposition numbers in both Houses to a unique percentage for both.
percentages = []
for i in range(len(df)):
    tot_seats = df['House Majority'][i] + df['House Opposition'][i] + df['Senate Majority'][i] + df['Senate Opposition'][i] 
    perc_m = int(round((df['House Majority'].iloc[i]+df['Senate Majority'].iloc[i])/tot_seats,2)*100)
    perc_o = int(round((df['House Opposition'].iloc[i]+df['Senate Opposition'].iloc[i])/tot_seats,2)*100)
    percentages.append([perc_m, perc_o])
perc_df = pd.DataFrame(percentages, columns=['Tot % Majority','Tot % Opposition'])
df = pd.concat([df.drop(['House Majority', 'House Opposition', 'Senate Majority', 'Senate Opposition'], axis=1), perc_df], axis=1)

#Computing the average of SDI between both Houses for Majority and Opposition.
dis = []
for i in range(len(df)):
    di_m = int(round((df['DI House Majority'].iloc[i]+df['DI Senate Majority'].iloc[i])/2,2)*100)
    di_o = int(round((df['DI House Opposition'].iloc[i]+df['DI Senate Opposition'].iloc[i])/2,2)*100)
    dis.append([di_m, di_o])
di_df = pd.DataFrame(dis, columns=['Tot DI Majority','Tot DI Opposition'])
df = pd.concat([df.drop(['DI House Majority', 'DI House Opposition', 'DI Senate Majority', 'DI Senate Opposition'], axis=1), di_df], axis=1)

#Transforming the SDI of government members into a integer.
df['DI Parties in Government'] = 100*df['DI Parties in Government']
df['DI Parties in Government'] = df['DI Parties in Government'].astype(int)

#Dropping the columns which represent the number of Vice-PM, Ministers with and without portfolio as their sum is included in the Tot. Gov. Members column
df = df.drop(['N° of VP', 'N° Ministers', 'N° Ministers without portfolio'], axis=1)
df['Tot Gov. Members'] = df['Tot Gov. Members'].astype(int)

#Dropping non-relevant data
df = df.drop(['Gov Name', 'End of Legislation', 'Time in Charge'], axis=1)

<h4>Pre-processing Activities</h4>
<div>As the dataset was kept as much broad as possible while designing it, in order to be reused in future projects, we have to perform some pre-processing activities to make it suitable for the definition of the Bayesian Network using the pgmpy library. In order to do so we have perform some initial simplification before we can start with the encoding part. This simplification will be necessary to create a more compact and easy to analyse model, at che cost of disregarding some potential causal links but with the objective of letting others emerge.
<ul>
    <li>The first simplification was the redeuction of the majority and opposition votes, these were converted by summing the two numbers for house and senate for both and then expressing them through the integer percentage they each represent.</li>
    <li>Then, a similar procedure was applied to the SDIs for Majority and Opposition which were replaced by their respective macro average for both majority and opposition.</li>
    <li>The SDI for government composition was converted into an integer to ease further discretization. </li>
    <li>The number of government members, being the sum of other three columns, will only be used as a measure of the size of the government, so its components were be dropped.</li>
    <li> Government Name was dropped as it is merely a nominal index.</li>
    <li> End of Legislation was dropped as it is already included in Fall Reason.</li>
    <li> As we chose to consider the period of actual activity of each governement, for the sake of determining the stability measure, we'll focus only on the Effecive Time in Charge, so Time in Charge was dropped.</li>
</ul>
    
</div>

In [4]:
#Encoding

#Replacing all Booleans with 0 or 1
df = df.replace({True:1, False:0})


#Replacing Fall Reason with number encoding
nominal_replacements = {
    'External Pressure':0,
    'Internal Struggle':1,
    'End of Legislation':2,
    'No confidence':3,
    'Lost Confidence':4,
    'Reshuffle':5,
}
df = df.replace(nominal_replacements)


#Encoding dictionary for non-boolean data
encode_dict = {
    'Effective Time in Charge': {'Intervals':[0,100,250,450,1000, 10000] ,'Labels':[0,1,2,3,4],},
    'Tot % Majority':{'Intervals':[0,50,55,66,100] ,'Labels':[0,1,2,3],},
    'Tot % Opposition':{'Intervals':[0,34,44,50,100] ,'Labels':[0,1,2,3],},
    'Tot DI Majority':{'Intervals':[-1,0,10,30,75,100] ,'Labels':[0,1,2,3,4],},
    'Tot DI Opposition':{'Intervals':[0,20,45,75,100] ,'Labels':[0,1,2,3],},
    'DI Parties in Government':{'Intervals':[-1,1,25,55,75,100] ,'Labels':[0,1,2,3,4],},
    'Tot Gov. Members':{'Intervals':[0,22,28,100] ,'Labels':[0,1,2],},
    'Swapped Ministers':{'Intervals':[-1,0,3,6,20] ,'Labels':[0,1,2,3],},
    'N° Majority Parties':{'Intervals':[0,1,4,15] ,'Labels':[0,1,2],},
    'N° Opposition Parties':{'Intervals':[0,1,4,15] ,'Labels':[0,1,2],},
    'First Year':{'Intervals':[0,1950, 1960,1970,1980, 1990,2000,2010,2020,2030] ,'Labels':[0,1,2,3,4,5,6,7,8],},
    'Legislation':{'Intervals':[0,12,50] ,'Labels':[1,2],},
}

for k in encode_dict.keys():
    #print(f"Encoding data {k} with intervals {encode_dict[k]['Intervals']} and labels:{encode_dict[k]['Labels']}")
    df[k] = pd.DataFrame(pd.cut(df[k].astype(int), encode_dict[k]['Intervals'], labels = encode_dict[k]['Labels']))

df = df.astype(int)
df.columns = ['Stability', 'Internal Changes', 'Decade', 'Republic Phase', 'Confidence',
             'Fall Cause', 'PM from Largest Majority Party', 'Technocrat/Independent PM', 'N° Majority Parties', 'N° Opposition Parties', 'Government Division',
             'Size of Government', 'Parliamentary Support', 'Parliamentary Opposition', 'Majority Division', 'Opposition Division']

<h4>Encoding</h4>
<div>
<p>Discretization is essential to highlight possible relationships between groups of values. In the case of the dataset presented each non-boolean value was reduced into a spceific set of ranges according to conventions or trying to give an ordered and rational separation of values in different categories.</p>
<b>Effective Time in Charge:</b> 
    <ul>
    <li>Unstable (0 to 100) ---> 0 </li>
    <li>Weak stability (101 to 250) ---> 1 </li>
    <li>Average stability (251 to 450) ---> 2 </li>
    <li>Solid stabiliy (451 to 1000) ---> 3 </li>
    <li>High stability (Over 1000) ---> 4 </li>
    </ul>


<b>Tot % Majority:</b>
    <ul>
    <li>No majority (0 to 50) ---> 0 </li>
    <li>Weak majorty (51 to 55) ---> 1 </li>
    <li>Strong majority (56 to 66) ---> 2 </li>
    <li>Constitutional majority (Over 66) ---> 3 </li>
    </ul>

<b>Tot % Opposition:</b>
    <ul>
    <li>Weak opposition (0 to 34) ---> 0 </li>
    <li>Medium opposition (35 to 44) ---> 1 </li>
    <li>Strong opposition (45 to 50) ---> 2 </li>
    <li>Potential new majority Over 50) ---> 3</li>
    </ul>

<b>Tot DI Majority:</b>
    <ul>
    <li>Single party majority (Only 0) ---> 0 </li>
    <li>Nearly one sided majority (1 to 10) ---> 1 </li>
    <li>Strong lead majority (11 to 30) ---> 2 </li>
    <li>Fragmented majority (31 to 75) ---> 3 </li>
    <li>Multi-headed majority (Over 76) ---> 4 </li>
    </ul>

<b>Tot DI Opposition:</b>
    <ul>
    <li>Cohesive Opposition (0 to 20) ---> 0 </li>
    <li>Diverse Opposition (21 to 45) ---> 1 </li>
    <li>Fragmented Opposition (46 to 75) ---> 2 </li>
    <li>Higly Fragmented/ Multi-Headed Opposition (Over 76) ---> 3 </li>
    </ul>

<b>DI Parties in Government:</b>
    <ul>
    <li>Monochromatic Government (Only 0) ---> 0 </li>
    <li>Governement with Leading Party (1 to 25) ---> 1 </li>
    <li>Coalition Government (26 to 55) --->  2 </li>
    <li>Large Coalition Government (56 to 75) ---> 3 </li>
    <li>Super Coalition/National Unity Government (Over 75) ---> 4 </li>
    </ul>
    
<b>Tot Gov. Members:</b>
    <ul>
    <li>Small Government (Under 22) ---> 0 </li>
    <li>Medium Government (23 to 28) ---> 1 </li>
    <li>Large Government (Over 29) ---> 2 </li>
    </ul>

<b>Swapped Ministers:</b>
    <ul>
    <li>Unchanged  (Only 0) ---> 0 </li>
    <li>Small Changes (1 or 2) ---> 1 </li>
    <li>Medium Changes (3 to 6) ---> 2 </li>
    <li>Large Changes (Over 6) ---> 3 </li>  
    </ul>

<b>N° Majority Parties:</b>
    <ul>
    <li>Single Party Majority (Only 1) ---> 0 </li>
    <li>Coalition Majority (2 to 4) ---> 1 </li>
    <li>Broad Coalition Majority (Over 5) ---> 2 </li>
    </ul>

<b>N° Opposition Parties:</b>
    <ul>
    <li>Single Party Opposition (Only 1) ---> 0  </li>
    <li>Differentiated Opposition (2 to 4) ---> 1 </li>
    <li>Fragmented Opposition (Over 5) ---> 2 </li>
    </ul>

<b>Fall Cause:</b>
    <ul>
    <li>External Pressure ---> 0 </li>
    <li>Internal Struggle ---> 1 </li>
    <li>End of Legislation ---> 2 </li>
    <li>No confidence ---> 3 </li>
    <li>Lost Confidence ---> 4 </li>
    <li>Reshuffle ---> 5 </li>
    </ul>

<b>First Year:</b>
<p>Starting years were split according to their decade and assigned a label from 0 (for the 40s) to 8 (for the 2020s).</p>

<b>Legislation:</b>
<p>As this information may have been redundant, given we are already considering the starting year, it was used to highlight a distinction between the two informally defined phases of the Italian Republic, labelled "First Republic" and "Second Republic", the second of which started with the 12th Legislation, when a completely different array of parties was elected in the Parliament. The distinction is then encoded with 1 for the First republic and 2 for the Second.</p>

<b>Boolean Values:</b>
<p>All boolean values were encoded as 0 for False and 1 for True.</p>
</div>

<b>Assigning new names to the columns</b>
<div>
    <p>As a consequence of the discretization some colums came to express a slight different meaning from the original, so they were renamed according to their new meaining:</p>
    <ul>
    <li>Effective Time in Charge ---> Stability </li>
    <li>Tot % Majority ---> Parliamentary Support</li>
    <li>Tot % Opposition ---> Parliamentary Opposition</li>
    <li>Tot DI Majority ---> Majority Division</li>
    <li>Tot DI Opposition ---> Opposition Division</li>
    <li>DI Parties in Government ---> Government Division</li>
    <li>Tot Gov. Members ---> Size of Government</li>
    <li>Swapped Ministers ---> Internal Changes</li>
    <li>N° Majority Parties ---> N° Majority Parties</li>
    <li>N° Opposition Parties ---> N° Opposition Parties</li>
    <li>First Year ---> Decade</li>
    <li>Legislation ---> Republic Phase</li>
    <li>Confidence ---> Confidence</li>
    <li>Fall Cause ---> Fall Cause</li>
    <li>PM from Largest Majority Party ---> PM from Largest Majority Party</li>
    <li>Technocrat/Independent PM ---> Technocrat/Independent PM</li>
    </ul>
</div>

In [5]:
#Building the Network

#Definition of the Causal Links
model_gov = BayesianNetwork(
    [('Decade', 'Republic Phase'),
     ('Decade', 'Opposition Division'),
     ('Decade', 'Majority Division'),
     ('Republic Phase', 'N° Majority Parties'),
     ('Republic Phase', 'N° Opposition Parties'),
     ('Majority Division', 'Parliamentary Support'),
     ('Majority Division', 'PM from Largest Majority Party'),
     ('Majority Division', 'Government Division'),
     ('N° Majority Parties', 'Parliamentary Support'),
     ('N° Majority Parties', 'Government Division'),
     ('N° Majority Parties', 'Majority Division'),
     ('N° Majority Parties', 'PM from Largest Majority Party'),
     ('N° Majority Parties', 'Technocrat/Independent PM'),
     ('N° Opposition Parties', 'Parliamentary Opposition'),
     ('N° Opposition Parties', 'Opposition Division'),
     ('Opposition Division', 'Parliamentary Opposition'),
     ('Parliamentary Support','Size of Government'),
     ('Parliamentary Support','Stability'),
     ('Parliamentary Support','Confidence'),
     ('Parliamentary Support','Technocrat/Independent PM'),
     ('Government Division', 'Stability'), 
     ('Government Division', 'Fall Cause'),
     ('Parliamentary Opposition', 'Stability'), 
     ('Parliamentary Opposition', 'Fall Cause'),    
     ('Size of Government', 'Internal Changes'),
     ('PM from Largest Majority Party','Confidence'),
     ('PM from Largest Majority Party', 'Stability'),
     ('Technocrat/Independent PM', 'Confidence'),
     ('Technocrat/Independent PM', 'Stability'),
     ('Internal Changes', 'Stability'),
     ('Internal Changes', 'Fall Cause'),
     ('Confidence', 'Fall Cause'),
    ]
)


<h4>Graphical representation of the network</h4>
<img src="Bayesian_Network_Final.png"/>

<h4>Building the Network</h4>
<p>The network was designed trying to draw the most important connections inside the available data:</p>
<div>
<ul>
<li><b>Decade</b> has a causal link to <b>Republic Phase</b> because the whole distinction between the journalistic terms "First Republic" and "Second Republic" hinges on the politcal turmoil of the early 90's which lead to great changes in the political party landscape. As a concequence, the decade we're considering higly influences if we're looking at the first or second republic period. </li>

<li><b>Decade</b> has a causal link to <b>Opposition Division</b> because across different years and decades the same set of parties may greatly change in terms of distributions of seats, leading to a more compact or fragmented opposition front.</li>

<li><b>Decade</b> has a causal link to <b>Majority Division</b> for symmetry with the one with Opposition Division as time passing may affect all political parties on both sides.</li>

<li><b>Republic Phase</b> has a causal link to <b>N° Majority Parties</b> because the passage itself between first and second republic can be attributed to a great change in the political parties landscape: as large parties dissolved in the early 90's we expect to see more parties in the second republic phase rather than the first. </li>

<li><b>Republic Phase</b> has a causal link to <b>N° Opposition Parties</b> for symmetry with the one with N° Majority Parties. </li>

<li><b>Majority Division</b> has a causal link to <b>Parliamentary Support</b> because the more diverse the majority a that supports a government is, the larger we expect the support to be as it maymean that many larger parties decided to ally to support the same government because none of them could support one by itself.</li>

<li><b>Majority Division</b> has a causal link to <b>Government Division</b> because a more diverse majority is more likely to support a governemnt which contains members from more parties, as each supporting party would claim one or more ministry depending on their strength. </li>

<li><b>Majority Division</b> has a causal link to <b>PM from Largest Majority Party/b> because a majority strongly led by one party is more probale to elect a PM from said party. </li>

<li><b>N° Majority Parties</b> has a causal link to <b>Parliamentary Support</b> because governments supported by lots of parties tend to receive a larger support than governments formed by fewer ones.</li>

<li><b>N° Majority Parties</b> has a causal link to <b>Government Division</b> because even if a party is the strongest one in a coalition, the presence of more parties may influence the number of ministers from other parties, thus increasing the diversity in government seats. </li>

<li><b>N° Majority Parties</b> has a causal link to <b>Majority Division</b> because a higher number of parties in the same coalition makes the coalition itself more diversified.</li>

<li><b>N° Majority Parties</b> has a causal link to <b>PM from Largest Majority Party</b> because fewer parties inside a coalition sould increas the probability that the PM will be chosen from the largest party inside the coalition. </li>

<li><b>N° Majority Parties</b> has a causal link to <b>Technocrat/Independent PM</b> because larger coalitions of parties, which are common to form in crisis situation, are more probable to be lead by an independent PM, in order to avoid conflicts between parties. </li>

<li><b>N° Opposition Parties</b> has a causal link to <b>Parliamentary Opposition</b> because a higher number of opposition parties rises the chances of having a larger opposition front.  </li>

<li><b>N° Opposition Parties</b> has a causal link to <b>Opposition Division</b> because as the nubmer of opposition parties grows it's more likely for ideologies to differ and for the opposition itself to be more fragmented.</li>

<li><b>Opposition Division</b> has a causal link to <b>Parliamentary Opposition</b> because in some cases, depending on the electroral law, a fragmented opposition may end up electing less seats than a unified one.</li>

<li><b>Parliamentary Support</b> has a causal link to <b>Size of Government</b> because larger support from the parliament come from the government promising to deal with multiple issues, which can be achieved by instituting new ministries, thus increasing the size of the government. </li>

<li><b>Parliamentary Support</b> has a causal link to <b>Stability</b> because a smaller support is more likely to be compromised by the actions of one party or few representatives or senators, thus leading to the fall of such government. </li>

<li><b>Parliamentary Support</b> has a causal link to <b>Confidence</b> because the entire concept of confidence hinges on the necessity of parliamentary support, minority governments are extrimely uncommon in Italian History. </li>

<li><b>Parliamentary Support</b> has a causal link to <b>Technocrat/Independent PM</b> because very large supports from the parliament are mostly achieved in crisis situations, when an independent PM is nominated in an attemp to appease the largest number of parties possible. </li>

<li><b>Government Division</b> has a causal link to <b>Stability</b> because a more fractured governemnt is more subject to internal constrasts, which makes it more probable to fall.</li>

<li><b>Government Division</b> has a causal link to <b>Fall Cause</b> because division inside the government coalition enhance the probabilities of the governemnt falling for internal struggles. </li>

<li><b>Parliamentary Opposition</b> has a causal link to <b>Stability</b> because a stronger opposition has higher chances of challenging the government enough to make it fall. </li>

<li><b>Parliamentary Opposition</b> has a causal link to <b>Fall Cause</b> because a stronger opposition enhances the probabilities of the government falling for external pressure. </li>

<li><b>Size of Government</b> has a causal link to <b>Internal Changes</b> because larger governement have more chances of being subject to changes. </li>

<li><b>PM from Largest Majority Party</b> has a causal link to <b>Confidence</b> because sometimes the imposition of a PM from the largest party results in the PM being denied confidence by the parliament.  </li>

<li><b>PM from Largest Majority Party</b> has a causal link to <b>Stability</b> because is considered more legitimated by the parliament if he comes from the largest party in the majority, leading to stability. On the other hand, as its choice was determined by the election results, its legitimacy may waver if recent polls amongst the citizens its belonging party lost its relevance.  </li>

<li><b>Technocrat/Independent PM</b> has a causal link to <b>Confidence</b> because independent/technocrat PMs are usually chosen when the state finds itself into a crisis situation or when it's attempting to solve a political stallmate, meaning that once the PMs has been chosen, its likely to gain confidence. </li>

<li><b>Technocrat/Independent PM</b> has a causal link to <b>Stability</b> because in case the independent/technocrat PM was chosen to lead the country through a crisis period, its more likely for its government to be stable. </li>

<li><b>Intenal Changes</b> has a causal link to <b>Stability</b> because many changes inside a governement may be a red flag for turmoil inside the government. </li>

<li><b>Intenal Changes</b> has a causal link to <b>Fall Cause</b> because many changes inside the government may forshadow the government falling for internal struggles. </li>

<li><b>Confidence</b> has a causal link to <b>Fall Cause</b> because a government which was denied confidence must fall beacuse of it.</li>

<li><b>Confidence</b> has a causal link to <b>Stability</b> because a government which was denied confidence shall fall immediately.</li>
</ul>
</div>

In [14]:
%%capture cap
# Fit the data to the Bayesian Network to obtain the CPTs
model_gov.cpds = []
model_gov.fit(df)

from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup


# Print all the learned CPTs
for cpd in model_gov.get_cpds():
    print('CPT of {}'.format(cpd.variable))
    print_full(cpd)
    print('\n')
with open('CPDs.txt', 'w', encoding='utf-8') as file:
    file.write(cap.stdout)


<h4>Fitting the Network</h4>
<div>
  The network was fit with all the values inside the dataset, we then print all the Conditional Probability Tables to have a quick overview of the results.  
</div>

In [7]:
#Queries
#We'll perform the queries using Variable Elimination
gov_inference_VE = VariableElimination(model_gov)
gov_inference_BP = BeliefPropagation(model_gov)

#Definition of Function to perform different types of Queries without considering encoding
label_encode_dict = {
    'Stability':{'Unstable':0, 'Weak stability':1, 'Average stability':2,'Solid stability':3,'High stability':4,},
    'Parliamentary Support':{'No majority':0, 'Weak majorty':1, 'Strong majority':2, 'Constitutional majority':3,},
    'Parliamentary Opposition':{'Weak opposition':0, 'Medium opposition':1, 'Strong opposition':2, 'Potential new majority':3,},
    'Majority Division':{'Single party majority':0, 'Nearly one sided majority':1, 'Strong lead majority':2, 'Fragmented majority':3, 'Multi-headed majority':4,},
    'Opposition Division':{'Cohesive Opposition':0, 'Diverse Opposition':1, 'Fragmented Opposition':2, 'Higly Fragmented/ Multi-Headed Opposition':3,},
    'Government Division':{'Monochromatic Government':0, 'Governement with Leading Party':1, 'Coalition Government':2, 'Large Coalition Government':3, 'Super Coalition/National Unity Government':4,},
    'Size of Government':{'Small Government':0, 'Medium Government':1,'Large Government':2,},
    'Internal Changes':{'Unchanged':0, 'Small Changes':1, 'Medium Changes':2, 'Large Changes':3,},
    'N° Majority Parties':{'Single Party Majority':0, 'Coalition Majority':1, 'Broad Coalition Majority':2,},
    'N° Opposition Parties':{'Single Party Opposition':0, 'Differentiated Opposition':1, 'Fragmented Opposition':2,},
    'Decade':{'40s':0,'50s':1,'60s':2,'70s':3,'80s':4,'90s':5,'2000s':6,'2010s':7,'2020s':8,},
    'Republic Phase':{'First Republic':1, 'Second Republic':2,},
    'Confidence':{'True':1,'False':0,},
    'Fall Cause':{'External Pressure':0, 'Internal Struggle':1, 'End of Legislation':2, 'No confidence':3, 'Lost Confidence':4, 'Reshuffle':5,},
    'PM from Largest Majority Party':{'True':1,'False':0,},
    'Technocrat/Independent PM':{'True':1,'False':0,},
}

label_decode_dict = {
    'Stability':{0:'Unstable', 1:'Weak stability', 2:'Average stability',3:'Solid stability',4:'High stability',},
    'Parliamentary Support':{0:'No majority', 1:'Weak majorty', 2:'Strong majority', 3:'Constitutional majority',},
    'Parliamentary Opposition':{0:'Weak opposition', 1:'Medium opposition', 2:'Strong opposition', 3:'Potential new majority',},
    'Majority Division':{0:'Single party majority', 1:'Nearly one sided majority', 2:'Strong lead majority', 3:'Fragmented majority', 4:'Multi-headed majority',},
    'Opposition Division':{0:'Cohesive Opposition', 1:'Diverse Opposition', 2:'Fragmented Opposition', 3:'Higly Fragmented/ Multi-Headed Opposition',},
    'Government Division':{0:'Monochromatic Government', 1:'Governement with Leading Party', 2:'Coalition Government', 3:'Large Coalition Government', 4:'Super Coalition/National Unity Government',},
    'Size of Government':{0:'Small Government', 1:'Medium Government',2:'Large Government',},
    'Internal Changes':{0:'Unchanged', 1:'Small Changes', 2:'Medium Changes', 3:'Large Changes',},
    'N° Majority Parties':{0:'Single Party Majority', 1:'Coalition Majority', 2:'Broad Coalition Majority',},
    'N° Opposition Parties':{0:'Single Party Opposition', 1:'Differentiated Opposition', 2:'Fragmented Opposition',},
    'Decade':{0:'40s',1:'50s',2:'60s',3:'70s',4:'80s',5:'90s',6:'2000s',7:'2010s',8:'2020s',},
    'Republic Phase':{1:'First Republic', 2:'Second Republic',},
    'Confidence':{1:'True',0:'False',},
    'Fall Cause':{0:'External Pressure', 1:'Internal Struggle', 2:'End of Legislation', 3:'No confidence', 4:'Lost Confidence', 5:'Reshuffle',},
    'PM from Largest Majority Party':{1:'True',0:'False',},
    'Technocrat/Independent PM':{1:'True',0:'False',},
}


def multi_query(model, variables, evidences, virtual_evidences=None, type='Direct', out=False):
     ev_dict = {}
     print(f">>>{type} Query for {variables} given the following hard evidences:\n")        
     for e in evidences.keys():
        ev_dict[e] = label_encode_dict[e][evidences[e]]
        print(f"{e} is {evidences[e]}")
     if type=='Direct':
        q = model.query(variables=variables, evidence=ev_dict, virtual_evidence=virtual_evidences)
     elif type=='Map':
        q = model.map_query(variables=variables, evidence=ev_dict, virtual_evidence=virtual_evidences)
     else:
        print(f'There is not such type of query as {type}')
        return

    
     if type=='Map':
        for k in variables:
            print(f"{k} is expected to be: {label_decode_dict[k][int(q[k])]}")
     else:
        print(q)

     if out:
        return q


<h4>Making some Queries</h4>

<p> Now that we fit the network we are ready to perform some queries on it, in order to perform them without the need to constantly refer to the encoding above a dictionary was defined to perform the conversion. An auxiliary function was also defined to enclose all query types into one function.</p>

<h5>Test Query</h5>
Before we start with actual predictive queries it's important to verify that our model can at least predict what should be obvious, so a test query needs to be performed. The chosen test is to predict that every government which was denied confidence fell for the same reason. This is an necessary outcome and one of the rare instances of an exact political process so it's a good candidate to be a test. 

In [8]:
#Test Query - Which is the Probability that a Government which was denied confidence to be unstable and fall because of it?
print("TEST QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['Stability','Fall Cause'], evidences={'Confidence':'False'}, type='Map')
print(60*"=" + '\n\n')

TEST QUERY==================================================
>>>Map Query for ['Stability', 'Fall Cause'] given the following hard evidences:

Confidence is False


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Stability is expected to be: Unstable
Fall Cause is expected to be: No confidence




<h6>Test Query - Comments</h6>
<p>From the test query we can clearly see that, assuming that the government was denied confidence, it's more probable for it to be extremly short lived and is certain to fall due to lack of confidence. This confirms the ability of the model to predict logical political relationships.</p>

<h5>First Query</h5>

<p>Given the fit model we now possess we may now try to make some predictions. As we have a model which can approximately predict the stability and fall cause for a government the first query which comes to mind is predict such values for the current government, which wasn't condidered in the dataset. So, after computing the values which influence the most stability and fall cause we can make a prediction.</p>

In [9]:
#First Query - What is the predicted stability of the current governement and how will it fall down?
print("FIRST QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['Stability','Fall Cause'], evidences={
    #'Decade':'2020s', #2024
    #'Republic Phase':'Second Republic',
    #'N° Majority Parties':'Coalition Majority', #4
    #'N° Opposition Parties':'Differentiated Opposition', #4
    #'Majority Division':'Fragmented majority', #63
    #'Opposition Division':'Fragmented Opposition', #71
    'Parliamentary Support':'Strong majority', #58
    'Parliamentary Opposition':'Medium opposition',#42 
    'Government Division': 'Super Coalition/National Unity Government',#81
    #'Size of Government': 'Medium Government',#27
    'PM from Largest Majority Party':'True', #True
    'Technocrat/Independent PM':'False', #False
    #'Internal Changes':'Unchanged', #0
    'Confidence':'True', #True
}, type='Map')
print(60*"=" + '\n\n')

FIRST QUERY==================================================
>>>Map Query for ['Stability', 'Fall Cause'] given the following hard evidences:

Parliamentary Support is Strong majority
Parliamentary Opposition is Medium opposition
Government Division is Super Coalition/National Unity Government
PM from Largest Majority Party is True
Technocrat/Independent PM is False
Confidence is True


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Stability is expected to be: Average stability
Fall Cause is expected to be: Reshuffle




<h6>First Query - Comments</h6>
<p>From what we can see, based on previous data, the current government is expected to have an average stability and to fall to perform a reshuffle of its components. Unfortunately for our model's prediction capability, the first outcome is not correct, as the current government already lasted more than 500 days, thus putting it at least in the Solid Stability category, but the second prediction may still become true as in the current political situation (as of 23/03/2024) there seems to be no alternative coalition which can effectively support another government, thus leaving two main options in case this government falls: either the same colaition agrees to support another, slightly modified government (causing a reshuffle) or the legislation will end and the country will have snap elections to elect a new parliament.  </p>

<h5>Second Query</h5>

<p>When presented with a situation of social, political or economical instability the President of the Republic may designate as Prime Minister a figure which comes from outside the political spectrum despite being well known for other achievements in governance-related fields. In other occasions, a similar phenomenon may be proposed by the parties themselves, proposing an independent figure to the President as a way to find a common ground between their agendas. In these case we end up with a Technocrat or and Independent PM. We now want to ask the model if this is an effective choice to obtain a stable government founded on a large parliamentary majority or if it is just a misconception.</p>

In [10]:
#Second Query - Is it true that, given a strong parliamentary support, a Technocrat-lead government is more stable than a Political one?
print("SECOND QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['Stability'], evidences={'Parliamentary Support':'Strong majority', 'Technocrat/Independent PM':'True',}, type='Map')
print('\n')
multi_query(model=gov_inference_VE, variables=['Stability'], evidences={'Parliamentary Support':'Strong majority', 'PM from Largest Majority Party':'True',}, type='Map')
print(60*"=" + '\n\n')

SECOND QUERY==================================================
>>>Map Query for ['Stability'] given the following hard evidences:

Parliamentary Support is Strong majority
Technocrat/Independent PM is True


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Stability is expected to be: Solid stability


>>>Map Query for ['Stability'] given the following hard evidences:

Parliamentary Support is Strong majority
PM from Largest Majority Party is True


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Stability is expected to be: Weak stability




<h6>Second Query - Comments</h6>

<p>We have two queries, in both we used as hard evidence the knowledge of strong parliamentary majority but in the first assumed a government led by a Technocrat, while in the second we assumed as PM a member of the largest majority party, and we tried to predict the stability of the government. From the results we can observe that our initial hypothesis wasn't a misconception as in the first case we end up with a predicted strong stability while in the second we have a weak one. This can be explained if we consider that a government founded on a large support may encounter more problems when it comes to clashes in political agendas between parties. If we suppose that an impartial leader has higher chances of solving such issues with respect to a politically aligned one, without encurring in a political crisis.</p>

<h5>Third Query</h5>

<p>Next we want to interrogate our model on political crisis scenario, it's reasonable to assume that, in situations where the country found itslef on a dire situation, the presence of a strong opposition against the government may hava caused some of its member parties to withdraw their support, thus leading to the government fall, usually through resignation of the PM. In these cases the external pressure, enchanced by the strength of the opposition used the internal division of the government as a leverage to make it fall. Supposing we want to find evidence for this claim we suppose as evidence the fall cause by external pressure and the presence of a strong opposition and we compute the CPT for government division.</p>

In [11]:
#Third Query - If we know that a government fell for external pressure and we know it had a strong opposition, what are the probabilities 
# of it being divided in its composition?
print("THIRD QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['Government Division'], evidences={'Fall Cause':'External Pressure', 'Parliamentary Opposition':'Strong opposition',}, type='Direct')
print(60*"=" + '\n\n')

THIRD QUERY==================================================
>>>Direct Query for ['Government Division'] given the following hard evidences:

Fall Cause is External Pressure
Parliamentary Opposition is Strong opposition
+------------------------+----------------------------+
| Government Division    |   phi(Government Division) |
+========================+============================+
| Government Division(0) |                     0.0472 |
+------------------------+----------------------------+
| Government Division(1) |                     0.0268 |
+------------------------+----------------------------+
| Government Division(2) |                     0.1315 |
+------------------------+----------------------------+
| Government Division(3) |                     0.6111 |
+------------------------+----------------------------+
| Government Division(4) |                     0.1834 |
+------------------------+----------------------------+




<h6>Third Query - Comments</h6>

<p>From what we can see in more than the 79% of the cases where there was a strong opposition and the government fell because of external pressure, we encounter a high division inside the government itself, this may suggest that a more politically diverse governement is more subject to opposition attacks which can compromise its integrity and lead to its eventual fall.</p>

<h5>Fourth Query</h5>

<p>We'll now focus on studying the optimal conditions under which a stable government can be formed, usually, when parties join forces to form a coalition there may be turmoils regarding the number of ministries to assign to each party, as no party with some influence would willingly renounce to its agenda. We'll focus on governments which reached the end of their legislation having a solid or high stability, we'll try to obtain the best coupling between majority division and government division.</p>

In [12]:
#Fourth Query - If we want a Government have at least a Solid Stability and reach the end of legislation, what are the best configurations for
# Majority Division and Government Division?
print("FOURTH QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['Majority Division', 'Government Division'], evidences={'Stability':'Solid stability', 'Fall Cause':'End of Legislation',}, type='Map')
multi_query(model=gov_inference_VE, variables=['Majority Division', 'Government Division'], evidences={'Stability':'High stability', 'Fall Cause':'End of Legislation',}, type='Map')
print(60*"=" + '\n\n')

FOURTH QUERY==================================================
>>>Map Query for ['Majority Division', 'Government Division'] given the following hard evidences:

Stability is Solid stability
Fall Cause is End of Legislation


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Majority Division is expected to be: Fragmented majority
Government Division is expected to be: Coalition Government
>>>Map Query for ['Majority Division', 'Government Division'] given the following hard evidences:

Stability is High stability
Fall Cause is End of Legislation


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Majority Division is expected to be: Fragmented majority
Government Division is expected to be: Large Coalition Government




<h6>Fourth Query - Comments</h6>

<p>From the obtained results we may make some reflections: first we can see that coalition and large coalition governments seem to be the most resilient, this may suggest that alliances between few parties with some common goals resist the most, assuming they overlook their differences. The idea of multiple points of view being a winning trait is also appearent in the other result about majority division, which is fragmented, suggesting that, if a fragmented majority succeeds in putting aside some of its differences, it may be capable of support the longest lived governments. This should't be too surprising as it's one of the basis for a parliamentary republic, which Italy is. Under this form of government it is expected for smaller parties to ally before or after every election instead of having large parties which single-handedly obtain a majority, like in the USA.</p>

<h5>Fifth Query</h5>

<p>The constitution is the base set of rules under which a whole nation if founded in the contemporary era. Each nation has, however, the possiblity of changing these rules, in order to adapt them. In Italy the reform of the constitution is an extraordinary procedure which requirese strict conditions in order to avoid degenerative reforms. In order to perform a revision of the constitution, it's mandatory to have a so called 'Constitutional Supermajority' of over 66% of the parliament in both houses or a simple majority vote followed by a popular referendum vote. Either way requires, however, a lot of time as this kind of reforms is usually quite difficult to pass, expecially using the direct supermajority approach. A possible option may be to join forces between multiple parties to agree upon the reforms. What we now want to ask the model is: assuming we obtain the required supermajority, we first want to know which are the best conditions under which it can form in terms of numbers of parties forming it and internal division. Then, using these parameters we'll compute the stability of such government and its predicted fall cause.  </p>

In [31]:
#Fifth Query - If a government aims at changing the constitution directly without consulting oppositions, how many parties should it contain,
# if we suppose it needs have a solid stability to perform the reform from beginning to end?
print("FIFTH QUERY" + 50*"=")
multi_query(model=gov_inference_VE, variables=['N° Majority Parties', 'Majority Division'], evidences={'Parliamentary Support':'Constitutional majority',}, type='Map')
multi_query(model=gov_inference_VE, variables=['Stability', 'Fall Cause'], evidences={'N° Majority Parties':'Broad Coalition Majority', 'Majority Division':'Fragmented majority',}, type='Map')
print(60*"=" + '\n\n')

FIFTH QUERY==================================================
>>>Map Query for ['N° Majority Parties', 'Majority Division'] given the following hard evidences:

Parliamentary Support is Constitutional majority


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

N° Majority Parties is expected to be: Broad Coalition Majority
Majority Division is expected to be: Fragmented majority
>>>Map Query for ['Stability', 'Fall Cause'] given the following hard evidences:

N° Majority Parties is Broad Coalition Majority
Majority Division is Fragmented majority


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Stability is expected to be: Weak stability
Fall Cause is expected to be: Internal Struggle




<h6>Fifth Query - Comments</h6>

<p>From the results we obtained we can dedue that the conditions for a direct constitutional reform are indeed hard to meet, as expected a supermajority would usually require the partecipation of the largest possible number of parties, making the whole coalition extremely fragmented and as a consequence of this, internal conflict arise more easily. This leads directly in the results of the second part of the query, which preicts a weak stability for such a government, which seems doomed to fall for internal struggles.</p>

<h4>Conclusions</h4>

<p>We now reached the end of our experiment and we can draw some conclusions. In this project a probabilistic model for analysing the political formation of Italian governements was presented and used to make some predictions, which, despite the many simplfications introduced, still turned out to be fairly resonable and this can be considered a success. During the development some of the relevant aspects for representing a country's political situation were omitted for the sake of simplicity but may still be integrated to obtain better results. Further imprvements may include the possibility of taking into account the effects of local election results on a national scale or, more geenrally, introduce the concept of political alignement inside the model, to better highlight different scenarios and improve predictions. </p>